### Load the dependencies


In [27]:
import os
import sys
import math
import psutil
os.environ["_CCTRACE_"]="ON"
# import cloudComPy as cc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
import tqdm.auto
from time import sleep, perf_counter
from pathlib import Path
from datetime import datetime
from multiprocessing import cpu_count
from computeclouds import ComputeClouds
print("Main : Libs loaded", datetime.now())

Main : Libs loaded 2022-07-12 15:25:56.183727


c:\Users\szinp\anaconda3\envs\CloudComPy39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Functions


In [2]:
def incrementParam(param_dic:dict, param_name:str, param_value:int or str) -> dict:
    global m3c2_params_dic
    m3c2_params_dic[param_name] = param_value


### TODO
```
- [ ] Add a function to bound box in txt
- [ ] Inrement paramters
- [ ] Connect functions
- [ ] Run the function 100 times


```

### Paths to files/ meshes/ pointClouds

##### TODO - add a functionality to lead from CLI

In [3]:
path1 = r"C:\Users\szinp\Desktop\GIS_PROJECT\lazorScan_project\testData\txtFiles\1_E_subsample.txt"
path2 = r"C:\Users\szinp\Desktop\GIS_PROJECT\lazorScan_project\testData\txtFiles\2_E_subsample.txt"

assert os.path.isfile(path1)
assert os.path.isfile(path2)
print("Paths are correct", datetime.now() )

Paths are correct 2022-07-12 14:04:02.898498


In [8]:
path1, path2 = ComputeClouds.createSampleCloud(2,4)

In [24]:
Xmin, Xmax, Ymin, Ymax, Zmin, Zmax = -0,3,-0,3,-10,10
boundingBox_1 = [Xmin, Xmax, Ymin, Ymax, Zmin, Zmax]
time_now = perf_counter()
with open(path1, "r") as f:
    data1 = f.readlines()
    with open(path2, "r") as f:
        data2 = f.readlines()
        print("Data loaded")
        print("Data1: ", len(data1))
        print("Data2: ", len(data2))


        # data1 = [list(map(float, line.split())) for line in data1]
        # data2 = [list(map(float, line.split())) for line in data2]

        data1_list = []
        data2_list = []
        for i in range(len(data1)):
            data1_list.append(data1[i].split(" "))
            data1_list[i] = [float(x) for x in data1_list[i]]
        for i in range(len(data2)):
            data2_list.append(data2[i].split(" "))
            data2_list[i] = [float(x) for x in data2_list[i]]

        print("data1_list: ", len(data1_list))
        print("data2_list: ", len(data2_list))

        print("data1_list: ", data1_list[:10])
        print("data2_list: ", data2_list[:10])
print("Execution for this funtion was",  perf_counter() - time_now)

np.savetxt(
    r"C:\Users\szinp\Desktop\GIS_PROJECT\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\dataFolder\pythonScripts_testInputs\1_E_subsample_ORIGINAL.txt",
    data1_list,
    delimiter=" ",
    newline="\n",
)
np.savetxt(
    r"C:\Users\szinp\Desktop\GIS_PROJECT\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\dataFolder\pythonScripts_testInputs\2_E_subsample_ORIGINAL.txt",
    data2_list,
    delimiter=" ",
    newline="\n",
)

Data loaded
Data1:  874466
Data2:  1000000
data1_list:  874466
data2_list:  1000000
data1_list:  [[-5.0, -5.0, 0.14142], [-5.0, -4.99, 0.141557], [-5.0, -4.98, 0.141666], [-5.0, -4.97, 0.141747], [-5.0, -4.96, 0.1418], [-5.0, -4.95, 0.141825], [-5.0, -4.94, 0.141822], [-5.0, -4.93, 0.14179], [-5.0, -4.92, 0.141731], [-5.0, -4.91, 0.141644]]
data2_list:  [[-5.0, -5.0, -0.001405], [-5.0, -4.99, 0.002595], [-5.0, -4.98, 0.006597], [-5.0, -4.97, 0.010598], [-5.0, -4.96, 0.014595], [-5.0, -4.95, 0.018584], [-5.0, -4.94, 0.022562], [-5.0, -4.93, 0.026526], [-5.0, -4.92, 0.030473], [-5.0, -4.91, 0.0344]]
Execution for this funtion was 9.011645900000076


In [21]:
# ********************************************************************************************************************
#! Filtering which is shitty
# ********************************************************************************************************************
time_now = perf_counter()

print("Filtering")
# to pop or not to pop
with tqdm.tqdm(total=len(data1_list)) as pbar:
    pbar.colour = "BLUE"
    for i in range(len(data1_list)):
        pbar.update(1)
        if (
            data1_list[i][0]    <   boundingBox_1[0]
            or data1_list[i][0] >   boundingBox_1[1]
            or data1_list[i][1] <   boundingBox_1[2]
            or data1_list[i][1] >   boundingBox_1[3]
            or data1_list[i][2] <   boundingBox_1[4]
            or data1_list[i][2] >   boundingBox_1[5]
        ):
            data1_list.pop(i)
            i -= 1
    pbar.close()

print("Filtering done")
print("data1_list: ", len(data1_list))
print("Execution for this funtion was",  perf_counter() - time_now)




Filtering


  3%|▎         | 26115/874466 [01:11<38:31, 367.03it/s]  


KeyboardInterrupt: 

In [28]:
# ********************************************************************************************************************
#? Filtering CHAD version
# ********************************************************************************************************************
time_now = perf_counter()

print("Filtering")

filtered_data1_list = []
filtered_data2_list = []
with tqdm.tqdm(total=len(data1_list)) as pbar:
    pbar.colour = 'MAGENTA'
    for cell_in_data1_list in data1_list:
        if cell_in_data1_list[0] > Xmax or cell_in_data1_list[0] < Xmin or cell_in_data1_list[1] > Ymax or cell_in_data1_list[1] < Ymin or cell_in_data1_list[2] > Zmax or cell_in_data1_list[2] < Zmin:
            continue
        else:
            filtered_data1_list.append(cell_in_data1_list)
        pbar.update(1)
    pbar.close()

with tqdm.tqdm(total=len(data2_list)) as pbar:
    pbar.colour = 'CYAN'
    for cell_in_data2_list in data2_list:
        # print("cell_in_data2_list: ", cell_in_data2_list)
        if cell_in_data2_list[0] > Xmax or cell_in_data2_list[0] < Xmin or cell_in_data2_list[1] > Ymax or cell_in_data2_list[1] < Ymin or cell_in_data2_list[2] > Zmax or cell_in_data2_list[2] < Zmin:
            continue
        else:
            filtered_data2_list.append(cell_in_data2_list)
        pbar.update(1)
    pbar.close()

print("Filtering done")
print("filtered_data1_list len: ", len(filtered_data1_list))
print("filtered_data2_list len: ", len(filtered_data2_list))
print("Execution for this funtion was",  perf_counter() - time_now)

np.savetxt(
    r"C:\Users\szinp\Desktop\GIS_PROJECT\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\dataFolder\pythonScripts_testInputs\1_E_subsample_testFunc2.txt",
    filtered_data1_list,
    delimiter=" ",
    newline="\n",
)

np.savetxt(
    r"C:\Users\szinp\Desktop\GIS_PROJECT\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\dataFolder\pythonScripts_testInputs\2_E_subsample_testFunc2.txt",
    filtered_data2_list,
    delimiter=" ",
    newline="\n",
)


Filtering


  9%|▉         | 90601/1000000 [00:01<00:10, 84420.95it/s] 


Filtering done
filtered_data1_list len:  90601
filtered_data2_list len:  90601
Execution for this funtion was 2.6883152999998856


## Load the data into CloudComPy module

In [ ]:
cloud1 = path1
cloud2 = path2

### Using M3C2 script to calculate the differences between the two clouds

In [ ]:
x,_,_,_,_ = [1,2,3,4,5]
print(x)

#### Creating parameters for M3C2 script

In [ ]:
m3c2_params_dic={}

m3c2_params_dic["M3C2VER"]= 1
m3c2_params_dic["NormalMode"]= 0
m3c2_params_dic["NormalScale"]= 10.691401
m3c2_params_dic["NormalMinScale"]= 5.3457
m3c2_params_dic["NormalStep"]= 5.3457
m3c2_params_dic["NormalMaxScale"]= 21.382801
m3c2_params_dic["SearchScale"]= 5.3457
m3c2_params_dic["SearchDepth"]= 41.898286
m3c2_params_dic["SubsampleRadius"]= 2.67285
m3c2_params_dic["NormalPreferedOri"]= 4
m3c2_params_dic["NormalUseCorePoints"]= "true"
m3c2_params_dic["SubsampleEnabled"]= "false"
m3c2_params_dic["RegistrationError"]= 0
m3c2_params_dic["RegistrationErrorEnabled"]= "false"
m3c2_params_dic["UseSinglePass4Depth"]= "false"
m3c2_params_dic["PositiveSearchOnly"]= "false"
m3c2_params_dic["UseMedian"]= "false"
m3c2_params_dic["UseMinPoints4Stat"]= "false"
m3c2_params_dic["MinPoints4Stat"]= 5
m3c2_params_dic["ProjDestIndex"]= 1
m3c2_params_dic["UseOriginalCloud"]= "false"
m3c2_params_dic["ExportStdDevInfo"]= "true"
m3c2_params_dic["ExportDensityAtProjScale"]= "true"
m3c2_params_dic["MaxThreadCount"]= 4
m3c2_params_dic["UsePrecisionMaps"]= "false"
m3c2_params_dic["PM1Scale"]= 1
m3c2_params_dic["PM2Scale"]= 1


Writing parameters to c:\Users\szinp\Desktop\GIS_PROJECT\CloudComPy39_20220513\CloudCompPy_Computing-CloudPoint\dataFolder\PythonParameters\m3c2_params.txt


In [ ]:
paramFilename =os.path.abspath("") + "\dataFolder\PythonParameters\m3c2_params.txt"
# paramFilename =os.path.abspath("") + r'\dataFolder\PythonParameters\bigData_1_m3c2_params.txt'
print(f"Writing parameters to {paramFilename}")
assert os.path.isfile(paramFilename), "File does not exist"

with open(paramFilename, 'w') as f:
    f.write("[General]\n")
    for key,value in m3c2_params_dic.items():
        f.write("%s=%s\n"%(key,value))

In [ ]:

paramFilename =os.path.abspath("") + "\dataFolder\PythonParameters\m3c2_params.txt"
# paramFilename =os.path.abspath("") + r'\dataFolder\PythonParameters\bigData_1_m3c2_params.txt'
print(f"Writing parameters to {paramFilename}")
assert os.path.isfile(paramFilename), "File does not exist"

with open(paramFilename, 'w') as f:
    f.write("[General]\n")
    for key,value in m3c2_params_dic.items():
        f.write("%s=%s\n"%(key,value))

In [ ]:
ComputeClouds.calcM3C2( cloud1, cloud2, paramFilename)